In [79]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("TheBloke/typhoon-7B-AWQ")

In [87]:
dataset = pd.read_csv('dataset.csv',encoding='utf-8')
dataset["source"] = dataset["source"].apply(lambda x: x.replace("\\","/"))
dataset.head(2)

,question,answer,references,source
0,หากมีการจ้างทำของเป็นการงานทำขึ้นบนอสังหาริมทร...,บุริมสิทธิที่เกิดจากการจ้างทำของเป็นการงานทำขึ...,อ้างอิงจากมาตรา 275 ของประมวลกฎหมายแพ่งและพาณิชย์,_ตรวจแล้ว/ประมวลกฎหมายแพ่งและพาณิชย์/ประมวลกฎห...
1,ในกรณีที่ผู้กระทำความผิดเป็นนิติบุคคล และความผ...,บุคคลนั้นต้องรับโทษตามที่บัญญัติไว้สำหรับความผ...,อ้างอิงจากมาตรา 42 ของพระราชบัญญัติว่าด้วยราคา...,_ตรวจแล้ว/พระราชบัญญัติว่าด้วยราคาสินค้าและบริ...


In [64]:
with open("test.txt", 'r') as f:
    raw = f.readlines()
    header = raw[0].strip()
    sections = "\n\n"+("".join(raw[1:]).strip())
    sections = sections.split("\n\nมาตรา")
# pattern = r'มาตรา\s?\d+'
sections = [f"มาตรา {section.strip()}" for section in sections]
if sections[0] == "มาตรา ": sections = sections[1:]
for i in range(3): print(">>",sections[i],"<<\n======\n")

>> มาตรา ภูมิพลอดุลยเดช ป.ร.
ให้ไว้ ณ วันที่ 30 พฤศจิกายน พ.ศ. 2497 เป็นปีที่ 9 ในรัชกาลปัจจุบัน

พระบาทสมเด็จพระปรมินทรมหาภูมิพลอดุลยเดช มีพระบรมราชโองการโปรดเกล้าฯ ให้ประกาศว่า
โดยที่เป็นการสมควรที่จะประกาศใช้ประมวลกฎหมายที่ดิน
จึงทรงพระกรุณาโปรดเกล้าฯ ให้ตราพระราชบัญญัติขึ้นไว้โดยคำแนะนำและยินยอมของสภาผู้แทนราษฎร ดังต่อไปนี้ <<

>> มาตรา 1  พระราชบัญญัตินี้เรียกว่า “พระราชบัญญัติให้ใช้ประมวลกฎหมายที่ดิน พ.ศ. 2497” <<

>> มาตรา 2  พระราชบัญญัตินี้ให้ใช้บังคับตั้งแต่วันถัดจากวันประกาศในราชกิจจานุเบกษาเป็นต้นไป <<



In [83]:
def get_knowledge(question,answer,ref,source,threshold=2000):
    with open(source, 'r') as f:
        raw = f.readlines()
        header = raw[0].strip()
        sections = "\n\n"+("".join(raw[1:]).strip())
        sections = sections.split("\n\nมาตรา")
    # pattern = r'มาตรา\s?\d+'
    sections = [f"มาตรา {section.strip()}" for section in sections]
    if sections[0] == "มาตรา ": sections = sections[1:]
    pattern = r'มาตรา\s?\d+(?:/\d+)?'
    keywords = list(set(re.findall(pattern, question) + re.findall(pattern, answer) + re.findall(pattern, ref)))

    extracted_content = {}

    # Iterate through keywords to find and extract relevant sections
    for keyword in keywords:
        # Compile a regex pattern to match the section starting with the keyword
        pattern = re.compile(r'^' + re.escape(keyword) + r' ', re.MULTILINE)
        
        for section in sections:
            if pattern.search(section):
                extracted_content[keyword] = section
                break  # Assuming each keyword uniquely identifies a section

    # Now, extracted_content contains the sections for the specified keywords
    if len(extracted_content) == 0:
        knowledges = "".join(raw)
    else:
        knowledges = f"{header}\n\n"+"\n\n".join(list(extracted_content.values()))
    # if len(keywords)>1:
    #     for key, value in extracted_content.items():
    #         print(f"{key}:\n{value}\n")
    #     print("\n=====================================\n")
    if len(tokenizer.tokenize(knowledges))>threshold:
        return False, knowledges
    return True, knowledges


    # if len(keywords)>1:
    #     return True, keywords
    # else: return False, keywords

In [89]:
knowledges = []
attention = []
tmp = dataset.copy()
for index, row in tqdm(tmp.iterrows()):
    question = row['question']
    answer = row['answer']
    ref = row['references']
    src = row['source']
    notMatches, legals = get_knowledge(question,answer,ref,src)
    knowledges.append(legals)
    attention.append(notMatches)
tmp["knowledges"] = knowledges
tmp[attention].to_csv('attention.csv',index=False,encoding='utf-8')
print(attention.count(False))

0it [00:00, ?it/s]

4121it [00:01, 2577.18it/s]


32


In [91]:
inverted_attention = [not i for i in attention]
tmp[inverted_attention].to_csv('attention_need_fix.csv',index=False,encoding='utf-8')

In [95]:
dataset_fixed = pd.read_csv('attention_fixed.csv',encoding='utf-8').reset_index(drop=True)
dataset_passed = pd.read_csv('attention.csv',encoding='utf-8').reset_index(drop=True)
dataset_new = pd.concat([dataset_fixed,dataset_passed],ignore_index=True)
dataset_new = dataset_new.sample(frac=1,random_state=2024).reset_index(drop=True)


In [96]:
dataset_new.head(5)

,question,answer,references,source,knowledges
0,ศาลมีอำนาจใดในการพิจารณาคดีเกี่ยวกับยาเสพติดตา...,ศาลมีอำนาจพิจารณาพิพากษาคดีโดยคำนึงถึงการสงเคร...,มาตรา 165 ของประมวลกฎหมายยาเสพติด,_ตรวจแล้ว/ประมวลกฎหมายยาเสพติด/ประมวลกฎหมายยาเ...,ประมวลกฎหมายยาเสพติด - ภาค 3 (บทกำหนดโทษ) - ลั...
1,เมื่อไหร่ที่สมาคมนายจ้างจะถือว่าเลิก,สมาคมนายจ้างย่อมเลิกด้วยเหตุใดเหตุหนึ่ง ดังต่อ...,มาตรา 82,_ตรวจแล้ว/พระราชบัญญัติแรงงานสัมพันธ์/พระราชบั...,พระราชบัญญัติแรงงานสัมพันธ์ (ฉบับที่ 3) พ.ศ. 2...
2,การยื่นอุทธรณ์ต้องทำภายในกี่เดือนหลังจากอ่านหร...,ต้องยื่นต่อศาลชั้นต้นในกำหนดหนึ่งเดือน,มาตรา 198 ของประมวลกฎหมายวิธีพิจารณาความอาญา,_ตรวจแล้ว/ประมวลกฎหมายวิธีพิจารณาความอาญา/ประม...,ประมวลกฎหมายวิธีพิจารณาความอาญา - ภาค 4 (อุทธร...
3,ศาลมีอำนาจใดบ้างเกี่ยวกับการรับฟังพยานหลักฐาน?,ศาลมีอำนาจปฏิเสธไม่รับพยานหลักฐานที่รับฟังไม่ไ...,อ้างอิงจากมาตรา 86 ของประมวลกฎหมายวิธีพิจารณาค...,_ตรวจแล้ว/ประมวลกฎหมายวิธีพิจารณาความแพ่ง/ประม...,ประมวลกฎหมายวิธีพิจารณาความแพ่ง - ภาค 1 (บททั่...
4,หากผู้เช่านาต้องการอุทธรณ์คำวินิจฉัยของคณะกรรม...,ผู้เช่านาต้องทำเป็นหนังสือยื่นต่อประธานคณะกรรม...,อ้างอิงจากมาตรา 56 ของพระราชบัญญัติการเช่าที่ด...,_ตรวจแล้ว/พระราชบัญญัติการเช่าที่ดินเพื่อเกษตร...,พระราชบัญญัติการเช่าที่ดินเพื่อเกษตรกรรม พ.ศ. ...


In [97]:
dataset_new.to_csv('dataset_v2.csv',index=False,encoding='utf-8')